In [2]:
import sympy as sp
import numpy as np
from sympy.physics.vector import dynamicsymbols
from sympy.physics.vector import time_derivative
from sympy.physics.vector import ReferenceFrame
N = ReferenceFrame('N')
import pylab as pl
import control
from EoM import *
from sympy.physics.mechanics import *
from numpy.linalg import matrix_rank, eig
import math
import intelligent_robotics as ir
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

In [33]:
h = 0.5

m1 = 2.486 + 0.3
m2 = 1.416
m3 = 1.739

L1 = 0.171
L2 = 0.28
L3 = 0.280

L1c = L1/2
L2c = L2 *0.75
L3c = L3 / 2

theta_1 = 150
DEG2RAD = np.pi/180
A = (h-L1*np.sin(theta_1*DEG2RAD))/L2
B = m2*L2c+m3*L2
C = m3*L3c
D = -(m1*L1c*np.cos(theta_1*DEG2RAD)+m2+L2*np.cos(theta_1*DEG2RAD))
E = (-L1*np.sin(theta_1*DEG2RAD))/L2

theta_2, theta_3 = sp.symbols('theta_2, theta_3')
f1 = sp.Eq(sp.sin(theta_2)+sp.sin(theta_3),float(A))
f2 = sp.Eq(float(B)*sp.cos(theta_2)+float(C)*sp.cos(theta_3),float(D))
# f3 = sp.Eq(sp.sin(theta_2)+sp.sin(theta_3),float(E))

sol = sp.solve([f1,f2])
sol

[]

In [10]:
theta_2, theta_3 = sp.symbols('theta_2, theta_3')
f1 = sp.Eq(sp.sin(theta_2)+sp.sin(theta_3),float(A))
f2 = sp.Eq(float(B)*sp.cos(theta_2)+float(C)*sp.cos(theta_3),float(D))
f3 = sp.Eq(sp.sin(theta_2)+sp.sin(theta_3),float(E))

sol = sp.solve([f1,f2,f3])
sol

In [5]:
h = 0.5
s = 0.01

m1 = 2.486 + 0.3
m2 = 1.416
m3 = 1.739
m4 = 16.09

L1 = 0.171
L2 = 0.28
L3 = 0.280
L4 = 0.346

L1c = L1/2
L2c = L2 *0.75
L3c = L3 / 2

A = (h-L4)/L2
B = s/L2

theta_2, theta_3 = sp.symbols('theta_2, theta_3')
f1 = sp.Eq(sp.cos(theta_2)+sp.cos(theta_3),float(A))
f2 = sp.Eq(sp.sin(theta_2)+sp.sin(theta_3),float(B))
# f3 = sp.Eq(sp.sin(theta_2)+sp.sin(theta_3),float(E))

sol = sp.solve([f1,f2])
sol

[{theta_2: -1.22676015168757, theta_3: 1.35644820679398},
 {theta_2: 1.35644820679398, theta_3: -1.22676015168757}]

In [5]:
L1c = L1
L2c = L2 *0.75
L3c = L3 / 2

D = -(m1*L1c*np.cos(135*DEG2RAD)+m2+L2*np.cos(135*DEG2RAD))

In [7]:
B = m2*L2c+m3*L2

In [ ]:
C = m3*L3c